In [1]:
import pandas as pd
import numpy as np
import requests
import pprint
import sidrapy
import warnings 
warnings.simplefilter("ignore")

# Estabelecer a conexão com a API do SIDRA IBGE

In [2]:
API = "https://apisidra.ibge.gov.br/values/t/1612/n3/43/v/allxp/p/last%2024/c81/2713"

Transformar o arquivo json em um Dataframe

In [3]:
req = requests.get(API)
info = req.json()

In [4]:
df = pd.DataFrame(info)
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)

Exluir colunas desnecessárias

In [5]:
df.drop(['Nível Territorial (Código)', 'Nível Territorial',
       'Unidade de Medida (Código)', 'Variável (Código)', 'Ano (Código)', 'Produto das lavouras temporárias (Código)',
       'Unidade de Medida'
       ],
       axis = 1, inplace = True)

Substituir valores não numéricos da coluna Valor e transformá-la em decimal

In [6]:
df.Valor.replace("-", np.nan, inplace = True)
df.Valor.replace("X", np.nan, inplace = True)
df.Valor.replace("...", np.nan, inplace = True)
df.Valor.replace("..", np.nan, inplace = True)

In [7]:
df.Valor = df.Valor.astype(float)

# Seperar os valores em linha e depois fazer o merge em colunas

In [8]:

area_plant = df.loc[df["Variável"] == "Área plantada"]
area_col = df.loc[df["Variável"] == "Área colhida"]
produtiv = df.loc[df["Variável"] == 'Rendimento médio da produção']
prod = df.loc[df["Variável"] == "Quantidade produzida"]
val = df.loc[df["Variável"] == "Valor da produção"]

In [9]:
area_plant.drop("Variável", axis = 1, inplace = True)
area_col.drop("Variável", axis = 1, inplace = True)
produtiv.drop("Variável", axis = 1, inplace = True)
prod.drop("Variável", axis = 1, inplace = True)
val.drop("Variável", axis = 1, inplace = True)

In [10]:
mer = area_plant.merge(area_col, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({
    'Valor_x': 'area_plantada(ha)',
    'Valor_y': 'area_colhida(ha)'
}, axis = 1, inplace = True)

Calcular a diferença entre áreas plantadas e colhidas

In [11]:
mer["plantada-colhida"] = mer['area_plantada(ha)'] - mer['area_colhida(ha)']

In [12]:
mer = mer.merge(prod, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "producao(t)"}, axis = 1, inplace = True)

In [13]:
mer = mer.merge(produtiv, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "produtividade(kg/ha)"}, axis = 1, inplace = True)

In [14]:
mer = mer.merge(val, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "valor_mil(R$)"}, axis = 1, inplace = True)

Calcular o valor médio da saca de soja

In [15]:
mer['valor_saca'] = (mer['valor_mil(R$)'] / ((mer['producao(t)'] / 60))).round(2)

In [16]:
mer.columns

Index(['area_plantada(ha)', 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Ano', 'Produto das lavouras temporárias',
       'area_colhida(ha)', 'plantada-colhida', 'producao(t)',
       'produtividade(kg/ha)', 'valor_mil(R$)', 'valor_saca'],
      dtype='object', name=0)

In [17]:
mer = mer[[ 'Ano', 'Unidade da Federação', 'Produto das lavouras temporárias', 'area_plantada(ha)',
       'area_colhida(ha)', 'plantada-colhida', 'producao(t)', 'produtividade(kg/ha)', 'valor_mil(R$)', 'valor_saca']]

In [18]:
mer.to_parquet("soja_IBGE_RS.parquet", index = False)

In [20]:
mer.head()

,Ano,Unidade da Federação,Produto das lavouras temporárias,area_plantada(ha),area_colhida(ha),plantada-colhida,producao(t),produtividade(kg/ha),valor_mil(R$),valor_saca
0,2000,Rio Grande do Sul,Soja (em grão),3030556.0,3001836.0,28720.0,4783895.0,1593.0,1385502.0,17.38
1,2001,Rio Grande do Sul,Soja (em grão),2976498.0,2974513.0,1985.0,6951830.0,2337.0,2381513.0,20.55
2,2002,Rio Grande do Sul,Soja (em grão),3307252.0,3295342.0,11910.0,5610518.0,1702.0,2477854.0,26.50
3,2003,Rio Grande do Sul,Soja (em grão),3591970.0,3591470.0,500.0,9579297.0,2667.0,5434779.0,34.04
4,2004,Rio Grande do Sul,Soja (em grão),3984337.0,3968530.0,15807.0,5541714.0,1396.0,3804582.0,41.19


# Importar as variáveis climátivas

# Upload do histórico da variação da temperatura média da superfícia do Oceano Pacífico

Os eventos de El Nino e La nina ocorrem quando há 5 ocorrências de temperaturas 0,5 acima (El Niño) ou abaixo (La Niña) do padrão. Os dados disponíveis nesse dataset

Estudos mostraram que uma condição necessária para o desenvolvimento e persistência da convecção profunda (aumento da nebulosidade e da precipitação) nos Trópicos é que a temperatura da superfície do mar (SST) local seja de 28 °C ou superior. Uma vez que o padrão de convecção profunda é alterado devido a SSTs anômalas, a circulação atmosférica tropical e subtropical se ajusta ao novo padrão de aquecimento tropical, resultando em padrões anômalos de precipitação e temperatura que se estendem muito além da região do Pacífico equatorial. Uma anomalia de SST de +0,5 °C na região do Niño 3.4 é suficiente para atingir esse limite entre o final de março e meados de junho. Durante o restante do ano, uma anomalia de SST maior, chegando a +1,5 °C nos meses de novembro, dezembro e janeiro, é necessária para atingir o limiar que sustenta a convecção profunda persistente nessa região.

In [23]:
enso = pd.read_excel("Temp_media_2010_2025.xlsx")

In [24]:
enso.head()

,Year,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,Media ultimo tri,Evento
0,2000,-1.5,-1.4,-1.1,-0.8,-0.7,-0.6,-0.6,-0.5,-0.5,-0.6,-0.7,-0.7,-0.600000,La nina
1,2001,-0.7,-0.5,-0.4,-0.3,-0.3,-0.1,-0.1,-0.1,-0.2,-0.3,-0.3,-0.3,-0.216667,Neutro
2,2002,-0.1,0.0,0.1,0.2,0.4,0.7,0.8,0.9,1.0,1.2,1.3,1.1,1.050000,El nino
3,2003,0.9,0.6,0.4,0.0,-0.3,-0.2,0.1,0.2,0.3,0.3,0.4,0.4,0.283333,Neutro
4,2004,0.4,0.3,0.2,0.2,0.2,0.3,0.5,0.6,0.7,0.7,0.7,0.7,0.650000,El nino


In [26]:
enso.rename(columns = {"Year": "Ano"}, inplace = True)

In [29]:
enso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ano               26 non-null     int64  
 1   Jan               26 non-null     float64
 2   Fev               26 non-null     float64
 3   Mar               26 non-null     float64
 4   Abr               26 non-null     float64
 5   Mai               26 non-null     float64
 6   Jun               25 non-null     float64
 7   Jul               25 non-null     float64
 8   Ago               25 non-null     float64
 9   Set               25 non-null     float64
 10  Out               25 non-null     float64
 11  Nov               25 non-null     float64
 12  Dez               25 non-null     float64
 13  Media ultimo tri  25 non-null     float64
 14  Evento            25 non-null     object 
dtypes: float64(13), int64(1), object(1)
memory usage: 3.2+ KB


In [34]:
mer["Ano"] = mer["Ano"].astype(int)

In [ ]:
mer = mer.merge(enso, on = "Ano", how = "left")

,Ano,Unidade da Federação,Produto das lavouras temporárias,area_plantada(ha),area_colhida(ha),plantada-colhida,producao(t),produtividade(kg/ha),valor_mil(R$),valor_saca,...,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,Media ultimo tri,Evento
0,2000,Rio Grande do Sul,Soja (em grão),3030556.0,3001836.0,28720.0,4783895.0,1593.0,1385502.0,17.38,...,-0.7,-0.6,-0.6,-0.5,-0.5,-0.6,-0.7,-0.7,-0.600000,La nina
1,2001,Rio Grande do Sul,Soja (em grão),2976498.0,2974513.0,1985.0,6951830.0,2337.0,2381513.0,20.55,...,-0.3,-0.1,-0.1,-0.1,-0.2,-0.3,-0.3,-0.3,-0.216667,Neutro
2,2002,Rio Grande do Sul,Soja (em grão),3307252.0,3295342.0,11910.0,5610518.0,1702.0,2477854.0,26.50,...,0.4,0.7,0.8,0.9,1.0,1.2,1.3,1.1,1.050000,El nino
3,2003,Rio Grande do Sul,Soja (em grão),3591970.0,3591470.0,500.0,9579297.0,2667.0,5434779.0,34.04,...,-0.3,-0.2,0.1,0.2,0.3,0.3,0.4,0.4,0.283333,Neutro
4,2004,Rio Grande do Sul,Soja (em grão),3984337.0,3968530.0,15807.0,5541714.0,1396.0,3804582.0,41.19,...,0.2,0.3,0.5,0.6,0.7,0.7,0.7,0.7,0.650000,El nino
5,2005,Rio Grande do Sul,Soja (em grão),4179272.0,3733822.0,445450.0,2444540.0,654.0,1161908.0,28.52,...,0.3,0.1,-0.1,-0.1,-0.1,-0.3,-0.6,-0.8,-0.333333,Neutro
6,2006,Rio Grande do Sul,Soja (em grão),3868501.0,3863726.0,4775.0,7559291.0,1956.0,2893470.0,22.97,...,-0.1,0.0,0.1,0.3,0.5,0.8,0.9,0.9,0.583333,El nino
7,2007,Rio Grande do Sul,Soja (em grão),3890903.0,3890183.0,720.0,9929005.0,2552.0,4518297.0,27.30,...,-0.4,-0.5,-0.6,-0.8,-1.1,-1.3,-1.5,-1.6,-1.150000,La nina
8,2008,Rio Grande do Sul,Soja (em grão),3804425.0,3803425.0,1000.0,7679939.0,2019.0,5496842.0,42.94,...,-0.8,-0.6,-0.4,-0.2,-0.2,-0.4,-0.6,-0.7,-0.416667,Neutro
9,2009,Rio Grande do Sul,Soja (em grão),3823246.0,3821936.0,1310.0,8025322.0,2099.0,5898631.0,44.10,...,0.0,0.3,0.5,0.6,0.7,1.0,1.4,1.6,0.966667,El nino
